In [63]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.io import read_image
from PIL import Image
import torch

# Define main dataset directory
dataset_dir = "C:/Users/vikas/OneDrive/Documents/Poacher_Detection_CNN/data"
poacher_dir = os.path.join(dataset_dir, "poacher")
non_poacher_dir = os.path.join(dataset_dir, "non_poacher")
csv_path = os.path.join(dataset_dir, "final_updated_data.csv")

# Initialize lists to store file paths and labels
file_paths = []
labels = []

# Function to collect image paths and labels from a directory
def collect_images(directory, label):
    if os.path.exists(directory):
        for filename in os.listdir(directory):
            if filename.lower().endswith((".jpg", ".png", ".jpeg")):
                file_paths.append(os.path.join(directory, filename))
                labels.append(label)

# Collect poacher and non-poacher images
collect_images(poacher_dir, "poacher")
collect_images(non_poacher_dir, "non-poacher")

# Check if the CSV already exists and append new entries
if os.path.exists(csv_path):
    existing_df = pd.read_csv(csv_path)
    new_df = pd.DataFrame({"file_path": file_paths, "label": labels})
    
    # Merge and drop duplicates to avoid re-adding existing entries
    combined_df = pd.concat([existing_df, new_df]).drop_duplicates(subset="file_path").reset_index(drop=True)
else:
    combined_df = pd.DataFrame({"file_path": file_paths, "label": labels})

# Save the combined DataFrame as CSV
combined_df.to_csv(csv_path, index=False)
print(f"CSV dataset updated at {csv_path}")

# Define the custom Dataset class
class PoacherDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        # Read the image as a tensor
        image = read_image(img_path)
        # Convert the tensor to a PIL Image
        image = transforms.ToPILImage()(image)
        # Convert image to RGB to handle inconsistent channels
        image = image.convert("RGB")
        # Encode labels as integers (1 for poacher, 0 for non-poacher)
        label = 1 if self.data.iloc[idx, 1].strip().lower() == "poacher" else 0
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations and load dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda image: image.convert("RGB")),  # Ensure all images are RGB
    transforms.ToTensor(),
])

# Instantiate the dataset and dataloader
dataset = PoacherDataset(csv_file=csv_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Verify a batch from the dataloader
for images, labels in dataloader:
    print(f"Batch of images shape: {images.shape}")
    print(f"Batch of labels: {labels}")
    break

# Define a simple CNN model for classification
import torch.nn as nn
import torch.optim as optim

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = x.view(x.size(0), -1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader):.4f}")

# Save the trained model
torch.save(model.state_dict(), 'poacher_detector.pth')

# Load the saved model for inference
model.load_state_dict(torch.load('poacher_detector.pth'))
model.eval()

# Prediction function
def predict_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    return "Poacher detected" if predicted.item() == 1 else "Non-poacher detected"

# Test the prediction
test_image_path = r"C:\Users\vikas\OneDrive\Documents\Poacher_Detection_CNN\data\poacher\IMG_2887.JPG"
result = predict_image(test_image_path)
print(result)


CSV dataset updated at C:/Users/vikas/OneDrive/Documents/Poacher_Detection_CNN/data\final_updated_data.csv
Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels: tensor([1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
        1, 0, 1, 1, 0, 1, 1, 0])
Epoch [1/10], Loss: 2.5566
Epoch [2/10], Loss: 0.4205
Epoch [3/10], Loss: 0.3506
Epoch [4/10], Loss: 0.2635
Epoch [5/10], Loss: 0.2153
Epoch [6/10], Loss: 0.1799
Epoch [7/10], Loss: 0.1363
Epoch [8/10], Loss: 0.1013
Epoch [9/10], Loss: 0.0743
Epoch [10/10], Loss: 0.0421
Poacher detected


C:\Users\vikas\AppData\Local\Temp\ipykernel_27816\891089453.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('poacher_detector.pth'))


In [64]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple CNN model for classification
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)  # First convolutional layer
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # Second convolutional layer
        self.fc1 = nn.Linear(32 * 56 * 56, 128)  # Fully connected layer (adjust size for your input dimensions)
        self.fc2 = nn.Linear(128, 2)  # Output layer with 2 classes: poacher and non-poacher

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))  # Apply ReLU activation after conv1
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)  # Apply max pooling
        x = nn.ReLU()(self.conv2(x))  # Apply ReLU activation after conv2
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)  # Apply max pooling
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        x = nn.ReLU()(self.fc1(x))  # Apply ReLU after fc1
        x = self.fc2(x)  # Final output layer
        return x


In [65]:
# Initialize the model, loss function, and optimizer
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()  # Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Optimizer with learning rate 0.001


In [66]:
# Define the number of epochs (number of times we go through the entire dataset)
num_epochs = 10  # Adjust as needed

# Loop over each epoch
for epoch in range(num_epochs):
    total_loss = 0  # To keep track of the total loss for the epoch
    for images, labels in dataloader:
        # Clear gradients before each step
        optimizer.zero_grad()

        # Forward pass: compute model output
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)
        
        # Backward pass: compute gradients
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        # Accumulate loss
        total_loss += loss.item()
    
    # Print the average loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss / len(dataloader):.4f}")


Epoch [1/10], Loss: 1.8674
Epoch [2/10], Loss: 1.2499
Epoch [3/10], Loss: 0.5666
Epoch [4/10], Loss: 0.5456
Epoch [5/10], Loss: 0.2754
Epoch [6/10], Loss: 0.2523
Epoch [7/10], Loss: 0.1609
Epoch [8/10], Loss: 0.1546
Epoch [9/10], Loss: 0.0939
Epoch [10/10], Loss: 0.0759


In [67]:
# Save the model's state dictionary to a file
torch.save(model.state_dict(), 'poacher_detector.pth')
print("Model saved as poacher_detector.pth")


Model saved as poacher_detector.pth


In [68]:
# Set the model to evaluation mode (disables dropout, etc.)
model.eval()

# Track correct predictions
correct = 0
total = 0

# Turn off gradients for faster computation and to save memory
with torch.no_grad():
    for images, labels in dataloader:  # Use test or validation dataloader here if available
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the index of the max log-probability
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')


Accuracy of the model on the test images: 98.82%


In [69]:
# Load the saved model
model = SimpleCNN()
model.load_state_dict(torch.load('poacher_detector.pth'))
model.eval()  # Set the model to evaluation mode


C:\Users\vikas\AppData\Local\Temp\ipykernel_27816\186802217.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('poacher_detector.pth'))


SimpleCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=100352, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [70]:
from PIL import Image
import torchvision.transforms as transforms

# Define the same transformations used in training
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to match the input size of the model
    transforms.ToTensor()           # Convert the image to a tensor
])

# Load and preprocess the image
def preprocess_image(image_path):
    image = Image.open(image_path)       # Open the image file
    image = transform(image)             # Apply transformations
    image = image.unsqueeze(0)           # Add batch dimension (1, C, H, W)
    return image


In [71]:
# Predict function
def predict_image(image_path):
    # Preprocess the image
    image = preprocess_image(image_path)
    
    # Forward pass: get the model output
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)  # Get the predicted class index (0 for non-poacher, 1 for poacher)

    # Map the prediction to the corresponding label
    if predicted.item() == 1:
        return "Poacher detected"
    else:
        return "Non-poacher detected"


In [72]:
# Path to the image you want to test
image_path = image_path = r"C:\Users\vikas\OneDrive\Documents\Poacher_Detection_CNN\data\poacher\IMG_2887.JPG"

# Run the detection
result = predict_image(image_path)
print(result)


Poacher detected
